In [1]:
import pyvista as pv
from seagullmesh import Mesh3
import numpy as np
import pymeshfix
from ga_population import *


pm = ProbeMesh.load(
    mesh_file=r"D:\mesh_extract_work\VervertMonkey\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Run_v4_mesh_extraction.ply",
    cam_pos=(-0.883204, -1.11311, 1.03748),
    tgt_pos=(0.062604, 0.152099, 0.187549),
    repair=True,
    fill_holes=False,
    cache=False,
    corpus_index='JF0N2N0B_VervetMonkeys_Run_v4_orig_Main.png',
)

C:\Users\dg\.conda\envs\diffnet4\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# mesh_file = r"D:\mesh_extract_work\VervertMonkey\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Run_v4_mesh_extraction.ply"
# pv_mesh = pv.read(mesh_file).triangulate().clean()

# meshfix = pymeshfix.MeshFix(pv_mesh.points, pv_mesh.regular_faces)
# meshfix.repair()
# pv_mesh = meshfix.mesh

# # pv_mesh.plot(show_edges=True)
# sm_mesh = Mesh3.from_pyvista(pv_mesh)

In [3]:
# mesh_reference_file = 'JF0N2N0B_VervetMonkeys_Run_v4_orig_Main.png'
# tgt_mesh = pv.read(r"D:\mesh_extract_work\VervertMonkey\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Blender\JF0N2N0B_VervetMonkeys_Run_v4_mesh_extraction.ply")
# tgt_mesh0 = tgt_mesh.triangulate().clean()
# cam_pos = (-0.883204, -1.11311, 1.03748)
# tgt_pos = (0.062604, 0.152099, 0.187549)

# mesh_reference_file = 'JC0L219A2_GreyTabbyCat_Trot_v4_orig_Main.png'
# tgt_mesh1 = pv.read(r"D:\mesh_extract_work\GreyTabbyCat\JC0L219A2_GreyTabbyCat_Blender\JC0L219A2_GreyTabbyCat_Blender\JC0L219A2_GreyTabbyCat_Trot_v4.mesh_extraction2.ply")

# mesh_reference_file = 'JU0L219A1_SnowshoeHare_Summer_Jump_v4_A_orig_Main.png'
# tgt_mesh = pv.read(r"D:\mesh_extract_work\SnowshoeHare\JU0L219A1_SnowshoeHare_Summer_Blender\JU0L219A1_SnowshoeHare_Summer_Jump_v4_A_mesh_extraction.ply")
# tgt_mesh2 = tgt_mesh.triangulate().clean()
# cam_pos = (-0.526595950126648, -0.8087201118469238, 0.40757232904434204)
# tgt_pos = (0.019130593165755272, -0.4574986398220062, 0.13628436625003815)

In [4]:
# meshfix = pymeshfix.MeshFix(tgt_mesh2.points, tgt_mesh2.regular_faces)
# meshfix.repair()
# sm = Mesh3.from_pyvista(meshfix.mesh)
# # diag = sm.bounding_box().diagonal()
#     # sm_wrapped = sm.alpha_wrapping(alpha=diag / 100, offset=diag / 600)

# # sm = Mesh3.from_pyvista(tgt_mesh2)
# # diag = sm.bounding_box().diagonal()
# # sm_wrapped = sm.alpha_wrapping(alpha=diag / 200, offset=diag / 600)
# # tgt_mesh2_wrapped = sm_wrapped.to_pyvista()

In [5]:
models = self = SourceModels.current_trained()

In [6]:
@dataclass
class VertexWeights:
    weights: np.ndarray  # (n_verts, n_channels)

def get_weights(self, probe_meshes: list[ProbeMesh], expt_kwargs=None):
    expt_kwargs = expt_kwargs or dict()

    n_model, n_probe = len(self), len(probe_meshes)
    weights = np.empty((n_probe, n_model), dtype='O')
    
    for model_idx, tm in enumerate(tqdm(self.models)):
        expt = tm.reader.experiment(**expt_kwargs)
        expt.model.eval()
        with torch.no_grad():
            for probe_idx, pm in enumerate(probe_meshes):
                _, preds, _ = expt.load_item(pm.mesh_data())
                weights[probe_idx, model_idx] = preds.cpu().numpy()

    return [
        VertexWeights(np.concatenate(weights[i, :], axis=1))
        for i in range(len(probe_meshes))
    ]
                    
weights, = get_weights(models, [pm], expt_kwargs=dict(outputs_at='vertices'))
weights

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:21<00:00,  3.66s/it]


VertexWeights(weights=array([[ 0.36066157,  0.34942096,  0.06562223, ...,  0.4627037 ,
         0.05185521,  0.06935756],
       [ 0.3311967 ,  0.34336364,  0.0644304 , ...,  0.46948028,
         0.05335634,  0.06879214],
       [ 0.3173782 ,  0.33545268,  0.06549861, ...,  0.47463843,
         0.0546254 ,  0.06766709],
       ...,
       [-0.0016617 ,  0.34771913,  0.03510686, ...,  0.54426813,
         0.18238369,  0.10073061],
       [ 0.25014418,  0.22300345,  0.09682684, ...,  0.4302503 ,
         0.02435485,  0.05456579],
       [ 0.51128983,  0.1305148 ,  0.08488552, ...,  0.52023965,
         0.04958928,  0.08333224]], dtype=float32))

In [7]:
weights.weights.shape

(2405, 86)

In [8]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
w_hat = pca.fit_transform(weights.weights)

In [9]:
w_hat_min = w_hat.min(axis=0)
w_hat_max = w_hat.max(axis=0)
w_hat_norm = (w_hat - w_hat_min) / (w_hat_max - w_hat_min)

pm.mesh.point_data['weights'] = w_hat_norm
# pm.mesh.plot(scalars='weights', rgb=True)

In [32]:
# pm.plot_weights(weights=w_hat_norm, shape=(1, 3))

def plot_weights(
        self,
        weights: np.ndarray,
        shape: tuple[int, int],
        plotter: pv.Plotter | None = None,
        link_views=True,
        render=True,
        scalar_bar=True,
        titles=None,
        **kwargs
):
    plotter = pv.Plotter(shape=(1, 3), off_screen=render, **kwargs)
    plotters = iter_subplots(plotter, shape=shape, link_views=link_views)
    if titles is None:
        titles = ['' for _ in range(np.prod(shape))]
    
    
    for i, ((row_col, p), ttl) in enumerate(zip(plotters, titles)):
        p.add_mesh(self.mesh.copy(), scalars=weights[:, i], show_scalar_bar=i==0 and scalar_bar)
        p.camera = self.camera

        if ttl:
            p.add_title(ttl)

    if render:
        return PIL.Image.fromarray(plotter.screenshot())  # noqa
    else:
        plotter.show()
        return plotter

plot_weights(pm, weights=w_hat_norm, shape=(1, 3), window_size=(1500, 500), scalar_bar=False, titles=['PC1', 'PC2', 'PC3'], render=False)

Widget(value='<iframe src="http://localhost:50080/index.html?ui=P_0x1c84d095d20_12&reconnect=auto" class="pyvi…

In [11]:
# for (i, j), p in iter_subplots(shape=(1, 3), link_views=True):
#     p.add_mesh(pm.mesh.copy(), scalars=w_hat_norm[:, j])

In [26]:
p = pv.Plotter()
p.add_title('asdsada')
# p.title = 'asdsad'
p.show()

Widget(value='<iframe src="http://localhost:50080/index.html?ui=P_0x1c8657eb820_8&reconnect=auto" class="pyvis…